### Initial Setup procedure

* Log in to GCP
* Start the free trial access
* Setup a billing account
* Create a Project
* Enable Google Cloud vision API and generate API token
https://cloud.google.com/vision/docs/before-you-begin
* Download Google SDK
* Run Google SDK -> Login -> Select Project
* Create a service account -> assign role -> download credential JSON

Codes - 
###### gcloud iam service-accounts create webdetection
###### gcloud projects add-iam-policy-binding web-detection-208318 --member "serviceAccount:webdetection@web-detection-208318.iam.gserviceaccount.com" --role "roles/owner"
###### gcloud iam service-accounts keys create webdetection_auth.json --iam-account webdetection@web-detection-208318.iam.gserviceaccount.com

* Upload JSON to Jupyter environment

In [1]:
import argparse
import io
import re
import pandas as pd
import numpy as np

from google.cloud import storage
from google.cloud import vision
from google.protobuf import json_format

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="webdetection_auth.json"

In [13]:
def detection(image):
    client = vision.ImageAnnotatorClient()

    # [START migration_web_detection]
    with io.open('nike_test.jpg', 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.web_detection(image=image)
    annotations = response.web_detection
    return annotations

In [14]:
# Converting web entities into dataframe
def web_entities(annotations):
    df2_entity_id = []
    df2_score = []
    df2_description = []
    if annotations.web_entities:
        for entity in annotations.web_entities:
            df2_entity_id.append(entity.entity_id)
            df2_score.append(entity.score)
            df2_description.append(entity.description)
    df_web_entities = pd.DataFrame(
        {'entity_id': df2_entity_id,
         'score': df2_score,
         'description': df2_description
        })
    df_labels = df_web_entities.description[df_web_entities.score > 0.50]
    df_labels = list(filter(None, df_labels)) 
    return df_labels 

In [15]:
#Convert best guess into list
def best_guess(annotations):
    df_best_guess = []
    if annotations.best_guess_labels:
        for label in annotations.best_guess_labels:
            df_best_guess.append(label.label)
    return df_best_guess

In [16]:
#Convert urls into dataframe
def urls(annotations):
    url = []
    if annotations.pages_with_matching_images:
        for page in annotations.pages_with_matching_images:
            url.append(page.url)
    df_url = pd.DataFrame(
        {'Weblinks':url})
    return df_url

In [17]:
def main(debug = False):
    #Process the image
    annotations = detection("nike_test.jpg")
    #find the labels
    df_labels = web_entities(annotations)
    print("The keywords are:", df_labels)
    #find the best guess of the product
    df_best_guess = best_guess(annotations)
    print("The best guess is:", df_best_guess)
    #find weblinks if any
    df_url = urls(annotations)
    df_amazon = []
    df_ebay = []
    df_pinterest = []
    df_amazon = df_url[df_url['Weblinks'].str.contains("amazon")==True]
    df_ebay = df_url[df_url['Weblinks'].str.contains("ebay")==True]
    df_pinterest = df_url[df_url['Weblinks'].str.contains("pinterest")==True]
    print("Amazon links:", df_amazon)
    print("Ebay links:", df_ebay)
    print("Pinterest links:" , df_pinterest)
    
if __name__ == "__main__":
    main()

The keywords are: ['Backpack', 'Nike', 'Bag', 'Jumpman', 'Duffel Bags', 'Sport']
The best guess is: ['nike girls school bags']
Amazon links: Empty DataFrame
Columns: [Weblinks]
Index: []
Ebay links: Empty DataFrame
Columns: [Weblinks]
Index: []
Pinterest links:                                             Weblinks
0  https://www.pinterest.com/veezpin/backpackduff...
1     https://www.pinterest.com/a_reynard/nike-bags/
2  https://www.pinterest.com.au/pin/5426838238265...
